In [98]:
import cv2 as cv
import numpy as np

print(cv.__version__)

4.7.0


In [99]:
class Detector:
    def __init__(self, fps):
        self.fps = fps
        self.T_f = self.fps * 5  # how many frames to be accumulated

        self.F_avg = 0
        self.F_sum = 0
        self.initial_stage = True
        self.initial_i = 2

        self.pre_frame = None
        self.cur_frame = None

        self.alpha = 1
        self.threshold_low = 72
        self.threshold_high = 168

    def __frame_preprocess(self, frame):
        f = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        f = cv.Canny(f, 100, 200)
        return f

    def pass_frame(self, frame):
        f = self.__frame_preprocess(frame)
        if self.cur_frame is None:
            self.cur_frame = f
            return 'initializing'
        else:
            self.pre_frame = self.cur_frame
            self.cur_frame = f

        if self.initial_stage == True:
            self.__initialize()
            return 'initializing'
        else:
            similarity = np.logical_and(self.pre_frame, self.cur_frame)
            similarity = np.sum(similarity)
            if similarity > self.F_avg / 3:
                return 'normal'

            if similarity > self.F_avg / 10:
                return 'occlusion'
            return self.__histogram_analyse()

    def __initialize(self):
        similarity = np.logical_and(self.pre_frame, self.cur_frame)
        similarity=np.sum(similarity)
        # if self.F_sum is None:
        #     self.F_sum = np.zeros(self.pre_frame.shape)
        self.F_sum += similarity
        self.initial_i += 1
        if self.initial_i >= self.T_f:
            self.F_avg = np.sum(self.F_sum / (self.T_f + 1))
            self.initial_stage = False

    def __histogram_analyse(self):
        alpha_motion = 1

        hist_cur = cv.calcHist(self.cur_frame, [0], None, [256], [0, 255])
        hist_pre = cv.calcHist(self.pre_frame, [0], None, [256], [0, 255])

        sum = np.sum(hist_cur[0:self.threshold_low + 1]) + np.sum(hist_cur[self.threshold_high:256])
        sum_pre = np.sum(hist_pre[0:self.threshold_low + 1]) + np.sum(hist_pre[self.threshold_high:256])
        alpha = sum_pre / sum

        # if alpha:
        return 'unknown'


print(cv.__version__)

4.7.0


In [100]:
# cap = cv.VideoCapture('datas/library_short.mp4')
# fps = int(cap.get(cv.CAP_PROP_FPS))
# detector = Detector(fps)
# result = []
# frame_i = 0
#
# while cap.isOpened():
#     ret, frame = cap.read()
#     if ret == False:
#         break
#     frame_i += 1
#     cur_time = frame_i // fps
#     detection = detector.pass_frame(frame)
#     if detection != 'normal' and detection != 'initializing':
#         result.append([cur_time, detection])
#
# cap.release()
# print(result)

In [101]:
cap = cv.VideoCapture('datas/library_short.mp4')
detector = Detector(fps=int(cap.get(cv.CAP_PROP_FPS)))

while cap.isOpened():
    ret, frame = cap.read()
    if ret == False:
        break

    frame_show = cv.resize(frame, (720, 720))  # 图像放缩

    detection = detector.pass_frame(frame)

    font = cv.FONT_HERSHEY_SIMPLEX
    cv.putText(frame_show, detection, (10, 100), font, 0.5, (255, 255, 0), 2)

    cv.imshow('frame', frame_show)
    c = cv.waitKey(25)
    if c == 27:  #ESC键
        break
cap.release()
cv.destroyAllWindows()